In [3]:
# import the necessary packages
import pandas as pd

In [4]:
# importing the data from the source files
workout_data = pd.read_csv('data/workout.csv')
three_keywords_data = pd.read_csv('data/three_keywords.csv')
workout_geo_data = pd.read_csv("data/workout_geo.csv")
three_keywords_geo_data = pd.read_csv("data/three_keywords_geo.csv")

In [5]:
# merging the data
workout_three_keywords_data = workout_data.merge(three_keywords_data, how='outer', on="month")
workout_three_keywords_data.head()

,month,workout_worldwide,home_workout_worldwide,gym_workout_worldwide,home_gym_worldwide
0,2018-03,59,12,16,10
1,2018-04,61,12,18,10
2,2018-05,57,13,16,9
3,2018-06,56,12,17,9
4,2018-07,51,12,17,9


In [8]:
# merging geographic data
geo_data = workout_geo_data.merge(three_keywords_geo_data, how="outer", on="Country")
geo_data.head()

# geo_data.shape


,Country,workout_2018_2023,home_workout_2018_2023,gym_workout_2018_2023,home_gym_2018_2023
0,Guam,NaN,NaN,NaN,NaN
1,Falkland Islands (Islas Malvinas),NaN,NaN,NaN,NaN
2,Cook Islands,NaN,NaN,NaN,NaN
3,Brunei,NaN,NaN,NaN,NaN
4,Palau,NaN,NaN,NaN,NaN


In [52]:
# combining the two datasets

combined_data = pd.concat([geo_data, workout_three_keywords_data], axis=1)

# cleaning the dataset
combined_data['workout_2018_2023'].fillna(combined_data['workout_2018_2023'].mean(), inplace=True)
combined_data['home_workout_2018_2023'].fillna(combined_data['home_workout_2018_2023'].mean(), inplace=True)
combined_data['gym_workout_2018_2023'].fillna(combined_data['gym_workout_2018_2023'].mean(), inplace=True)
combined_data['home_gym_2018_2023'].fillna(combined_data['home_gym_2018_2023'].mean(), inplace=True)
combined_data['workout_worldwide'].fillna(combined_data['workout_worldwide'].mean(), inplace=True)
combined_data['home_workout_worldwide'].fillna(combined_data['home_workout_worldwide'].mean(), inplace=True)
combined_data['gym_workout_worldwide'].fillna(combined_data['gym_workout_worldwide'].mean(), inplace=True)
combined_data['home_gym_worldwide'].fillna(combined_data['home_gym_worldwide'].mean(), inplace=True)

combined_data["year"] = combined_data["month"].str[:4]
combined_data['year'].ffill(inplace=True)
combined_data.drop(inplace=True, columns=["month"])
print(combined_data['year'].unique())
print(combined_data.shape)
# combined_data.isna().sum()

combined_data.head()

['2018' '2019' '2020' '2021' '2022' '2023']
(250, 10)


,Country,workout_2018_2023,home_workout_2018_2023,gym_workout_2018_2023,home_gym_2018_2023,workout_worldwide,home_workout_worldwide,gym_workout_worldwide,home_gym_worldwide,year
0,Guam,27.967213,40.787234,34.212766,25.0,59.0,12.0,16.0,10.0,2018
1,Falkland Islands (Islas Malvinas),27.967213,40.787234,34.212766,25.0,61.0,12.0,18.0,10.0,2018
2,Cook Islands,27.967213,40.787234,34.212766,25.0,57.0,13.0,16.0,9.0,2018
3,Brunei,27.967213,40.787234,34.212766,25.0,56.0,12.0,17.0,9.0,2018
4,Palau,27.967213,40.787234,34.212766,25.0,51.0,12.0,17.0,9.0,2018


In [57]:
# Selecting columns for worldwide data
worldwide_data = combined_data[
    [
        "year",
        "workout_worldwide",
        "home_workout_worldwide",
        "gym_workout_worldwide",
        "home_gym_worldwide",
    ]
].copy()

# Selecting columns for country-wise data
country_data = combined_data[
    [
        "year", "Country",
        "workout_2018_2023",
        "home_workout_2018_2023",
        "gym_workout_2018_2023",
        "home_gym_2018_2023",
    ]
].copy()

In [58]:
# Exporting the cleaned datasets to CSV files
worldwide_data.to_csv("cleaned_worldwide_fitness_data.csv", index=False)
country_data.to_csv("cleaned_country_fitness_data.csv", index=False)

In [54]:
# Export the cleaned dataset to a sqlite db file
import sqlite3

conn = sqlite3.connect("cleaned_combined_fitness_data.sqlite")
combined_data.to_sql("fitness_data", con=conn, if_exists='replace', index=False)

250